# XGBoost Model

## Method explanation

The `xgboost` module is a popular implementation of tree boosting. One of its main differences with the regular boosting introduced in class is the presence of a regularization term directly in the objective function to minimize. The following discussion is based on [XGBoost's documentation](https://xgboost.readthedocs.io/en/stable/index.html) and [Chen, T., & Guestrin, C. (2016, August). Xgboost: A scalable tree boosting system. In Proceedings of the 22nd acm sigkdd international conference on knowledge discovery and data mining (pp. 785-794).](https://arxiv.org/pdf/1603.02754.pdf)

We recall that as we have seen the final boosted model is a sum of $B$ trees which can be written as

$$\hat{f}(x) = \sum_{b=1}^B \hat{f}^b(x)$$

In classical boosting, during training the model is minimizing

$$\mathcal{L}(\hat{f}) = \sum_{i=1}^n \eta L(\hat{y}_i, y_i) \quad \quad \text{where } \hat{y}_i = \hat{f}(x_i)$$

Here, $\eta \in (0,1)$ is the shrinkage parameter and $L$ is the loss function. For regression, this is typically the squared error loss, whereas for classification, this could be the maximum likelihood loss. XGBoost, on the other hand, introduces a second term in the objective to minimize, which can then be rewritten as

$$\mathcal{L}(\hat{f}) = \sum_{i=1}^n \eta L(\hat{y}_i, y_i) + \sum_{b=1}^B \Omega(\hat{f}^b) \quad \quad
\text{where }\Omega(f) = \gamma T + \dfrac{1}{2} \lambda ||w||^2$$

The parameters used in the regularization function $\Omega$ are listed below:
- $T$ is the number of leaves of the tree. This penalizes bigger leaf numbers, which means trees which are more complex.
- $w \in \mathbb{R}^T$ are the weights which are assigned in each leaf of the tree. The weights are the predicted outputs in each leaf. They are real values if we have a regression tree, and probabilities if we have a decision tree.
- $\gamma \geq 0$ and $\lambda \geq 0$ are penalty terms, akin to parameter $\lambda$ in ridge or lasso linear regression, for example. The bigger their values, the stronger the regularization is. They are tuning parameters.


The implementation of regularization directly in the objective function helps to avoid overfitting which can happen easily when using boosting methods. XGBoost also provides a lot of other improvements from a point of view of computational efficiency. We will not get in the details as their are more directly related to computer science, however some aspects will be alluded to later when discussing all the tuning parameters we have used to build our model.

## Preliminary steps

First, we import the necessary modules.

In [438]:
import pandas as pd
import numpy as np
from scipy import stats
import csv

from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier

We import the data and split it into predictors `X` and response `y`.

In [439]:
campaign_ad = pd.read_csv("MLUnige2023_subscriptions_train.csv", index_col="Id")
campaign_test = pd.read_csv("MLUnige2023_subscriptions_test.csv", index_col="Id")

X = campaign_ad.drop(columns='subscription')
y = campaign_ad['subscription']

XGBoost can handle categorical variables, but their `dtype` must be converted to `category`. We do so below.

In [440]:
# numerical variables
num_vars = ['age', 'time_spent', 'banner_views', 'banner_views_old', 'days_elapsed_old', 'X4']
# categorical variables
cat_vars = list(set(X.columns).difference(num_vars))

# converting type to category
for col in cat_vars:
  X[col] = X[col].astype("category")
  campaign_test[col] = campaign_test[col].astype("category")

We split `X` and `y` into training, validation and test sets. The seeds are indentical to the ones used with other models to ensure that we have a fair ground.

In [441]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=46)

We define our `xgb_clf` below with the following parameters:
- `tree_method` is set to `'hist'`. One of the particularities implemented by XGBoost is that when choosing the next best split for a tree, instead of sorting all values of each predictor and go through them to determine where the optimal split should occur, the algorithm will create an histogram which approximates the distribution of the predictor. This makes the whole process much more fast.
- `objective` is set to `'binary:logistic'`. This is the appropriate objective function when dealing with a classification task in which the outome variable is binary.
- `eval_metric` is set to `'auc'`. In classification, a commonly used visualization to assess model performance is the ROC curve, which plots the True Positive Rate against the False Positive Rate. A model which performs well should have a curve which increases as steeply and quickly as possible. Thus, a possible measure used to assess the model performance is the area under the ROC curve, which is called AUC. A bigger area means that the model is performing better. The AUC is handy, as it does not depend on the choice of probablility cutoff for classifying records as 0 or 1.
- `enable_categorical` is set to `True`. This is necessary if there are categorical predictors, so that XGBoost correctly handles them.
- `n_jobs` is set to `-2`. This parameter enambles portions of the algorithm to be run in parallel on multiple cores of the CPU.

In [442]:
xgb_clf = XGBClassifier(tree_method='hist',
                        objective='binary:logistic',
                        eval_metric='auc',
                        enable_categorical=True,
                        n_jobs=-2,
                        random_state=391)

Now, we define a parameter grid for our parameter search. We explain the tuning parameters which are used below:
- `n_estimators` is the number of trees $B$.
- `max_depth` is the maximum depth allowed for all trees.
- `learning_rate` is the shrinkage parameter $\eta$.
- `gamma` is the penalty parameter $\gamma$ defined inside regularization function $\Omega$.
- `reg_lambda` is the penalty parameter $\lambda$ defined inside regularization function $\Omega$.

We also incorporated the two following parameters in early grid searches, but their ideal values were remaining close the the default ones, so we decided to keep them are their default.
- `colsample_bytree` is the proportion of the total features which are used to grow each tree. For example, if it is equal to 0.5, it means that for each tree, only half the predictors are randomly picked among all possible features. Thus, each tree has its own combination of predictors. By default, it is set to 1.
- `scale_pos_weight` is the balancing of weights. By default, it is set to 1, which means that when considering the response `subscription`, 1 and 0 are weighted symmetrically. If the ratio is bigger than 1, more importance is given to the positive class, and vice versa.

In [443]:
param_search = {'n_estimators': range(50, 301),
                'max_depth': range(1, 8),
                'learning_rate': stats.uniform(0.001, 0.3),   # [0.001, 0.301]
                'colsample_bytree': stats.uniform(0.5, 0.5),  # [0.5, 1]
                'gamma': stats.uniform(0, 0.5),               # [0, 0.5]
                'reg_lambda': stats.uniform(0, 5)             # [0, 5]
                }

In the above grid, for continuous tuning parameters, we used a uniform distribution inside an interval instead of a list of values. This is because we are going to use `RandomizedSearchCV` instead of the classical grid search. This gridsearch will generate random values from the distributions we provided. To do so, in each fold it will go through a number `n_iter` of iterations taken from the grid we constructed. We have chosen to use the `StratifiedKFold` function which creates folds which all have a similar proportion of positive responses.

In [444]:
model = RandomizedSearchCV(xgb_clf,
                           param_distributions=param_search,
                           n_iter=250,
                           scoring='roc_auc',
                           return_train_score=True,
                           verbose=1,
                           cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=611),
                           random_state=37)

model.fit(X_train, y_train)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=611, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=True,
                                           eval_metric='auc',
                                           feature_types=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importa...
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd8a0a8c0d0>,
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd8a8a350a0>,
                                        'max_depth': range(1, 8),
                                        'n_estimators': range(50, 301),
                                        'reg_lambda': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd8a2362e50>},
                   random_state=37, return_train_score=True, scoring='roc_auc',
                   verbose=1)

We can now retrieve the best parameters from our search.

In [445]:
best_params = model.best_params_
best_params

{'colsample_bytree': 0.635562621004793,
 'gamma': 0.4165486523112224,
 'learning_rate': 0.04689135859111069,
 'max_depth': 7,
 'n_estimators': 261,
 'reg_lambda': 3.8384092245326618}

In [342]:
{'colsample_bytree': 0.9605195575049101,
 'gamma': 0.2780067362138148,
 'learning_rate': 0.16146905484916832,
 'max_depth': 6,
 'n_estimators': 114,
 'scale_pos_weight': 1}

We can also look at the accuracy of the best 

In [446]:
model.best_score_

0.9316474777096093

In [447]:
xgb_model = XGBClassifier(**best_params,
                          tree_method='hist',
                          objective='binary:logistic',
                          eval_metric='auc',
                          enable_categorical=True)

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.635562621004793, early_stopping_rounds=None,
              enable_categorical=True, eval_metric='auc', feature_types=None,
              gamma=0.4165486523112224, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.04689135859111069, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=261, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [448]:
train_pred = xgb_model.predict_proba(X_train)[:,1]
valid_pred = xgb_model.predict_proba(X_valid)[:,1]

In [449]:
cutoff_table = pd.DataFrame({'cutoff': 1e-2*np.arange(10, 95, 5)})
cutoff_table['train_acc'] = [np.round(accuracy_score(y_train, (train_pred > cutoff).astype(int)), 3)
                             for cutoff in cutoff_table['cutoff']]
cutoff_table['valid_acc'] = [np.round(accuracy_score(y_valid, (valid_pred > cutoff).astype(int)), 3)
                             for cutoff in cutoff_table['cutoff']]
cutoff_table

,cutoff,train_acc,valid_acc
0,0.10,0.838,0.792
1,0.15,0.874,0.812
2,0.20,0.902,0.830
3,0.25,0.922,0.838
4,0.30,0.935,0.846
5,0.35,0.947,0.851
6,0.40,0.954,0.847
7,0.45,0.959,0.850
8,0.50,0.964,0.845
9,0.55,0.965,0.847


In [450]:
cutoff_table = pd.DataFrame({'cutoff': 1e-2*np.arange(30, 65, 1)})
cutoff_table['train_acc'] = [np.round(accuracy_score(y_train, (train_pred > cutoff).astype(int)), 4)
                             for cutoff in cutoff_table['cutoff']]
cutoff_table['valid_acc'] = [np.round(accuracy_score(y_valid, (valid_pred > cutoff).astype(int)), 4)
                             for cutoff in cutoff_table['cutoff']]
cutoff_table

,cutoff,train_acc,valid_acc
0,0.30,0.9354,0.8459
1,0.31,0.9373,0.8466
2,0.32,0.9397,0.8474
3,0.33,0.9416,0.8474
4,0.34,0.9437,0.8496
5,0.35,0.9473,0.8511
6,0.36,0.9485,0.8511
7,0.37,0.9497,0.8526
8,0.38,0.9513,0.8526
9,0.39,0.9526,0.8481


Choosing cutoff

In [451]:
treshold = 0.5

In [452]:
y_test_pred = (xgb_model.predict_proba(X_test)[:,1] > treshold).astype(int)
accuracy_score(y_test, y_test_pred)

0.8607594936708861

In [453]:
xgb_model.fit(X, y)
y_REAL_test = (xgb_model.predict_proba(campaign_test)[:,1] > treshold).astype(int)

In [454]:
y_pred = (xgb_model.predict_proba(X)[:,1] > treshold).astype(int)
accuracy_score(y, y_pred)

0.9521894548704201

In [341]:
file = open('test_file_xgboost.csv', 'w')
writer = csv.writer(file)
writer.writerow(['Id', 'subscription'])
for i in range(len(y_REAL_test)):
    writer.writerow([i, y_REAL_test[i]])
file.close()

## Ensemble attempt

In [71]:
y_pred_xgboost = pd.read_csv('test_file_xgboost.csv')
y_pred_boosting = pd.read_csv('test_file_boost.csv')
y_pred_NN = pd.read_csv('test_file_neuralnets.csv')

In [92]:
file = open('test_file_ensemble.csv', 'w')
writer = csv.writer(file)
writer.writerow(['Id', 'subscription'])
for i in range(len(y_pred_xgboost)):
    pred = int(np.round(np.mean([y_pred_boosting.loc[i,'subscription'],
                                 y_pred_NN.loc[i,'subscription'],
                                 y_pred_xgboost.loc[i,'subscription']])))
    writer.writerow([i, pred])
file.close()